<a href="https://colab.research.google.com/github/RandyTunru/FashionMNIST_Classification/blob/main/Fashion_MNIST_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import torch
from torchvision import datasets
from torchvision.models import resnet
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torchvision.transforms import ToTensor

In [ ]:
np.random.seed(2510)
torch.manual_seed(2510)

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.0MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 203kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.76MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 13.0MB/s]


In [ ]:
print(training_data)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
type(train_dataloader)

torch.utils.data.dataloader.DataLoader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

fc_model = NeuralNetwork().to(device)
print(fc_model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
class ConvolutionModel(nn.Module):
    def __init__(self, n_channels, n_class):
        super().__init__()
        self.flatten = nn.Flatten()
        self.conv_stack1 = nn.Sequential(
            nn.Conv2d(in_channels=n_channels, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        conv_output_size = (28 - 3 + 2 * 1) // 1 + 1
        max_pool_output_size = (conv_output_size - 2) // 2 + 1

        self.conv_stack2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        conv_output_size = (max_pool_output_size - 3 + 2 * 1) // 1 + 1
        max_pool_output_size = (conv_output_size - 2) // 2 + 1


        with torch.no_grad():
          dummy = torch.ones(1, n_channels, 28, 28)
          dummy = self.conv_stack1(dummy)
          dummy = self.conv_stack2(dummy)
          dummy = self.flatten(dummy)
          flatten_dim = dummy.view(-1).shape[0]


        self.linear_relu_stack = nn.Sequential(
            nn.Linear(flatten_dim, 512),
            nn.ReLU(),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
        )



        self.classifier = nn.Linear(512, n_class)

    def forward(self, x):
        x = self.conv_stack1(x)
        x = self.conv_stack2(x)
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        logits = self.classifier(x)
        return logits

custom_cnn_model = ConvolutionModel(n_channels=1, n_class=10).to(device)
print(custom_cnn_model)

ConvolutionModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (conv_stack1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_stack2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
  )
  (classifier): Linear(in_features=512, out_features=10, bias=True)
)


In [ ]:
resnet_18_model = resnet.resnet18(num_classes=10).to(device)

resnet_18_model.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
resnet_18_model.maxpool = nn.Identity()

nn.init.kaiming_normal_(resnet_18_model.conv1.weight, mode="fan_out", nonlinearity="relu")

resnet_18_model = resnet_18_model.to(device)

print(resnet_18_model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), p

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train(dataloader, model, loss_fn):
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
def inference(dataloader, model):
    model.eval()
    pred = None
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)

    return pred

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, fc_model, loss_fn)
    test(test_dataloader, fc_model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304441  [   64/60000]
loss: 2.297501  [ 6464/60000]
loss: 2.268801  [12864/60000]
loss: 2.264300  [19264/60000]
loss: 2.242896  [25664/60000]
loss: 2.209267  [32064/60000]
loss: 2.224827  [38464/60000]
loss: 2.183050  [44864/60000]
loss: 2.179285  [51264/60000]
loss: 2.150638  [57664/60000]
Test Error: 
 Accuracy: 42.6%, Avg loss: 2.140314 

Epoch 2
-------------------------------
loss: 2.146119  [   64/60000]
loss: 2.144505  [ 6464/60000]
loss: 2.073084  [12864/60000]
loss: 2.100970  [19264/60000]
loss: 2.028572  [25664/60000]
loss: 1.971592  [32064/60000]
loss: 2.010378  [38464/60000]
loss: 1.917474  [44864/60000]
loss: 1.924077  [51264/60000]
loss: 1.863693  [57664/60000]
Test Error: 
 Accuracy: 57.9%, Avg loss: 1.851349 

Epoch 3
-------------------------------
loss: 1.875571  [   64/60000]
loss: 1.859306  [ 6464/60000]
loss: 1.725130  [12864/60000]
loss: 1.782429  [19264/60000]
loss: 1.646529  [25664/60000]
loss: 1.613898  [32064/600

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, custom_cnn_model, loss_fn)
    test(test_dataloader, custom_cnn_model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303164  [   64/60000]
loss: 2.301698  [ 6464/60000]
loss: 2.300458  [12864/60000]
loss: 2.296741  [19264/60000]
loss: 2.299154  [25664/60000]
loss: 2.293104  [32064/60000]
loss: 2.288279  [38464/60000]
loss: 2.286369  [44864/60000]
loss: 2.289739  [51264/60000]
loss: 2.280201  [57664/60000]
Test Error: 
 Accuracy: 25.8%, Avg loss: 2.284006 

Epoch 2
-------------------------------
loss: 2.285421  [   64/60000]
loss: 2.285273  [ 6464/60000]
loss: 2.278732  [12864/60000]
loss: 2.277332  [19264/60000]
loss: 2.275336  [25664/60000]
loss: 2.266762  [32064/60000]
loss: 2.263536  [38464/60000]
loss: 2.256625  [44864/60000]
loss: 2.260794  [51264/60000]
loss: 2.243544  [57664/60000]
Test Error: 
 Accuracy: 37.5%, Avg loss: 2.246774 

Epoch 3
-------------------------------
loss: 2.252177  [   64/60000]
loss: 2.249072  [ 6464/60000]
loss: 2.229937  [12864/60000]
loss: 2.228146  [19264/60000]
loss: 2.207518  [25664/60000]
loss: 2.191470  [32064/600

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, resnet_18_model, loss_fn)
    test(test_dataloader, resnet_18_model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.394788  [   64/60000]
loss: 2.122333  [ 6464/60000]
loss: 1.937794  [12864/60000]
loss: 1.833347  [19264/60000]
loss: 1.581503  [25664/60000]
loss: 1.498508  [32064/60000]
loss: 1.391025  [38464/60000]
loss: 1.246103  [44864/60000]
loss: 1.177441  [51264/60000]
loss: 1.095626  [57664/60000]
Test Error: 
 Accuracy: 71.3%, Avg loss: 1.065360 

Epoch 2
-------------------------------
loss: 1.074306  [   64/60000]
loss: 1.047838  [ 6464/60000]
loss: 0.783777  [12864/60000]
loss: 1.034013  [19264/60000]
loss: 0.816175  [25664/60000]
loss: 0.856520  [32064/60000]
loss: 0.881770  [38464/60000]
loss: 0.779812  [44864/60000]
loss: 0.818281  [51264/60000]
loss: 0.748260  [57664/60000]
Test Error: 
 Accuracy: 76.0%, Avg loss: 0.737374 

Epoch 3
-------------------------------
loss: 0.709584  [   64/60000]
loss: 0.773141  [ 6464/60000]
loss: 0.514966  [12864/60000]
loss: 0.775091  [19264/60000]
loss: 0.637151  [25664/60000]
loss: 0.647146  [32064/600